In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [16]:
df = pd.read_csv("train.csv", header=None)
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols

features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

df

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.619,-0.536,-0.092,0.182,0.034,-0.172,0.401,0.393,0.216,1
1,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,-2.568,0.034,NaN,0.401,1.890,0.216,1
396,NaN,NaN,NaN,V3,V2,NaN,NaN,V1,V3,V2,...,NaN,NaN,NaN,-4.401,-2.684,NaN,0.401,1.391,0.216,1
397,NaN,NaN,NaN,V3,V2,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,0.548,4.926,NaN,0.401,0.992,0.216,1
398,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V8,...,NaN,NaN,NaN,-1.651,3.567,NaN,0.401,0.992,0.216,1


In [17]:
print((np.count_nonzero(target == 1)/np.count_nonzero(target == -1)) * 100)

68.77637130801688


In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#TODO Experiment with strategies
iterative_numerical_imputer = IterativeImputer(max_iter=10)
iterative_numerical_imputer.fit(X_train[numerical_features])
X_train.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_train.loc[:, numerical_features])
X_test.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_test.loc[:, numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(X_train[categorical_features])
X_train.loc[:, categorical_features] = categorical_imputer.transform(X_train.loc[:, categorical_features])
X_test.loc[:, categorical_features] = categorical_imputer.transform(X_test.loc[:, categorical_features])

C:\Users\BALU\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"
C:\Users\BALU\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\BALU\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [20]:
X_train_len = len(X_train)
X = pd.concat([X_train, X_test])
X = pd.get_dummies(X)
X_train = X[:X_train_len]
X_test = X[X_train_len:]

In [21]:
#X.to_csv("save\X.csv", index=False)
#X_train.to_csv("save\X_train.csv", index=False)
#X_test.to_csv("save\X_test.csv", index=False)

In [22]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	#model.fit(X_train, y_train)
	return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
#models.append(SVC(gamma='auto'))
#names.append('SVM')
# KNN
#models.append(KNeighborsClassifier())
#names.append('KNN')
# Neutral Network
#models.append(MLPClassifier(solver = 'adam', max_iter=1000, tol=0.000001, early_stopping=True, validation_fraction=0.1, n_iter_no_change=20))
#names.append('NEU')
# Bagging
models.append(BaggingClassifier(n_estimators=1000))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000))
names.append(' ET')


results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))
	

>BAG 0.803 (0.037)
> RF 0.808 (0.049)
> ET 0.824 (0.050)


In [23]:
from sklearn.model_selection import GridSearchCV

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Create the random grid
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 1000, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': max_depth,
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

print(random_grid)

{'n_estimators': [1000, 1111, 1222, 1333, 1444, 1555, 1666, 1777, 1888, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [24]:
import time

rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, search across 100 different combinations, and use all available cores
grid_search = GridSearchCV(estimator = rf, param_grid = random_grid, cv = 5, n_jobs = -1, verbose = 2)
#rf_random = (estimator = rf, param_distributions = random_grid, n_iter = 1000, cv = 4, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
start = time.time()

grid_search.fit(X_train, y_train)

end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

Fitting 5 folds for each of 4320 candidates, totalling 21600 fits
Elapsed time:  11701.365000247955  seconds, which is:  195.02275000413258  minutes


In [25]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 60,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1111}

In [26]:
def evaluate(model, X_test, y_test):
    score = model.score(X_test, y_test)
    print('Accuracy = {:0.2f}%.'.format(score))
    return score
    
base_model = RandomForestClassifier(n_estimators = 1000)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

random_accuracy = evaluate(grid_search.best_estimator_, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Accuracy = 0.80%.
Accuracy = 0.86%.
Improvement of 7.81%.


In [27]:
# Predicting the original Test Set
test_to_predict = pd.read_csv("test.csv", header=None)

numerical_features = test_to_predict.select_dtypes(include=['float64']).columns
categorical_features = test_to_predict.select_dtypes(include=['object']).columns

In [28]:
test_to_predict.loc[:, numerical_features] = iterative_numerical_imputer.transform(test_to_predict.loc[:, numerical_features])
test_to_predict.loc[:, categorical_features] = categorical_imputer.transform(test_to_predict.loc[:, categorical_features])

test_to_predict_len = len(test_to_predict)
original = pd.concat([features, test_to_predict])
original = pd.get_dummies(original)
test_to_predict = X[:test_to_predict_len]

In [29]:
test_to_predict

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,C12_V1,C12_V2,C12_V3,C13_V1,C13_V2,C13_V3,C13_V4,C13_V5,C14_V1,C14_V2
104,-1.663000,0.875,0.823,0.736,-0.070,-0.513,-0.099000,-0.844000,0.994,0.568000,...,0,0,1,0,1,0,0,0,0,1
369,0.165547,0.627,0.348,-0.986,0.447,-0.513,0.997000,-0.008958,-0.071,-0.014618,...,0,0,1,1,0,0,0,0,0,1
329,-0.004000,0.875,-1.092,-0.282,-0.458,-0.513,-1.196000,0.689000,0.842,-0.546000,...,1,0,0,1,0,0,0,0,1,0
295,2.484000,-0.118,-1.567,0.892,-0.716,0.779,-1.196000,1.800000,-0.071,-1.445000,...,1,0,0,1,0,0,0,0,0,1
248,-1.110000,-0.986,1.312,-1.846,-0.458,0.133,0.038775,-0.844000,0.537,1.234000,...,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,-0.557000,-1.359,0.348,-0.360,-0.458,-0.513,-0.319000,-0.216000,0.537,-0.438000,...,1,0,0,1,0,0,0,0,1,0
232,0.272000,0.379,0.348,-1.064,-1.750,-0.513,-0.648000,1.507000,-0.984,-1.985000,...,1,0,0,1,0,0,0,0,1,0
80,-0.557000,-0.242,-0.617,-0.047,-0.587,1.425,0.339000,-0.844000,0.537,1.234000,...,1,0,0,1,0,0,0,0,1,0
165,-1.110000,0.503,3.226,-0.282,-1.104,-1.159,0.120000,-0.844000,0.385,0.568000,...,1,0,0,1,0,0,0,0,0,1


In [30]:
prediction = grid_search.best_estimator_.predict(test_to_predict)
print("Count(1): ", np.count_nonzero(prediction == 1), "Count(-1): ", np.count_nonzero(prediction == -1))
print((np.count_nonzero(prediction == 1)/np.count_nonzero(prediction == -1)) * 100)

Count(1):  34 Count(-1):  66
51.515151515151516


In [31]:
# Exporting
prediction = pd.DataFrame(prediction)
prediction.to_csv("predictions.csv", index=False, header=False)

In [32]:
# TODO https://datascience.stackexchange.com/questions/75345/need-help-understanding-data-leakage
# TODO https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74